![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/dodkins/PythonProjects/stocks/options_testing/options_testing')

import os
from datetime import datetime 
import pandas as pd
import opstrat as op

/Users/dodkins/.conda/envs/stocks/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from qc_simulator import QuantBook, Resolution, OptionRight

In [3]:
QuantBook()

In [4]:
import quantconnect
from utils import aggregate, concat_dfs, get_start_price
from technical_analysis import get_poc
# from strategies import measure_period_profit
# from strategies import add_expirations

In [5]:
qbw = quantconnect.QuantBookWrapper({'qb':QuantBook(),'Resolution':Resolution,'OptionRight':OptionRight})

In [6]:
qb = QuantBook()
tsla = qb.AddEquity("TSLA")
equity_symbol = tsla.Symbol

In [7]:
qb.History(qb.Securities.Keys, 3000, Resolution.Hour)

open        high         low       close  \
symbol time                                                                  
lol    2020-12-30 10:00:00  224.026644  225.026644  222.799978  224.404978   
       2020-12-30 11:00:00  224.409978  225.669977  223.693311  224.606644   
       2020-12-30 12:00:00  224.619978  227.833311  224.596644  227.166644   
       2020-12-30 13:00:00  227.136644  228.999977  226.999977  228.066644   
       2020-12-30 14:00:00  228.033311  229.816644  227.666644  229.376644   
...                                ...         ...         ...         ...   
       2022-09-13 12:00:00  293.680000  293.990000  290.400000  291.000000   
       2022-09-13 13:00:00  291.000000  293.280000  290.670000  292.300000   
       2022-09-13 14:00:00  292.270000  295.500000  291.780000  294.830000   
       2022-09-13 15:00:00  294.830000  295.970000  293.000000  294.100000   
       2022-09-13 16:00:00  294.120000  294.990000  291.400000  292.130000   

                                volume  
symbol time                             
lol    2020-12-30 10:00:00  12666775.0  
       2020-12-30 11:00:00  16660004.0  
       2020-12-30 12:00:00  21427847.0  
       2020-12-30 13:00:00  15279809.0  
       2020-12-30 14:00:00  12887422.0  
...                                ...  
       2022-09-13 12:00:00   7883773.0  
       2022-09-13 13:00:00   5298142.0  
       2022-09-13 14:00:00   9703434.0  
       2022-09-13 15:00:00   7136008.0  
       2022-09-13 16:00:00  10745308.0  

[3000 rows x 5 columns]

In [8]:
from strategies import IronCondors, measure_period_profit
import plots

In [9]:
ic_strat = IronCondors(long_offset=15, short_offset=15, qbw=qbw)

In [10]:
tsla = qbw.get_tsla()

In [11]:
ic_strat = IronCondors(long_offset=15, short_offset=15, qbw=qbw)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='week',
    update_freq='once')

In [12]:
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [13]:
concise_cols = ['date', 'underlying_open', 'underlying_close', 'hourly_profit', 'running_profit', 'new_option', 'start_price', 'sell_call_strike', 'buy_call_strike', 'sell_put_strike', 'buy_put_strike', 'sell_call_strike_open', 'sell_call_strike_close', 'buy_call_strike_open', 'buy_call_strike_close', 'sell_put_strike_open', 'sell_put_strike_close', 'buy_put_strike_open', 'buy_put_strike_close']
ic[concise_cols]

,date,underlying_open,underlying_close,hourly_profit,running_profit,new_option,start_price,sell_call_strike,buy_call_strike,sell_put_strike,buy_put_strike,sell_call_strike_open,sell_call_strike_close,buy_call_strike_open,buy_call_strike_close,sell_put_strike_open,sell_put_strike_close,buy_put_strike_open,buy_put_strike_close
0,2022-08-03 13:00:00,304.746636,307.993303,-0.002100,-0.002100,True,304.746636,351.0,357.0,261.0,255.0,0.002235,0.005456,-0.000489,-0.001299,5.476585e-04,2.003592e-04,-5.485496e-05,-1.781226e-05
1,2022-08-03 14:00:00,307.943303,307.446636,0.000872,-0.001228,False,304.746636,351.0,357.0,261.0,255.0,0.004870,0.004257,-0.001132,-0.000977,1.755880e-04,2.058429e-04,-1.505614e-05,-1.798100e-05
2,2022-08-03 15:00:00,307.473303,307.983303,-0.000098,-0.001327,False,304.746636,351.0,357.0,261.0,255.0,0.003859,0.004440,-0.000866,-0.001010,1.756369e-04,1.487841e-04,-1.474151e-05,-1.224767e-05
3,2022-08-03 16:00:00,307.993303,307.396636,0.000778,-0.000549,False,304.746636,351.0,357.0,261.0,255.0,0.004002,0.003387,-0.000890,-0.000741,1.266361e-04,1.542405e-04,-1.000481e-05,-1.247093e-05
4,2022-08-04 10:00:00,311.236636,311.539969,-0.000112,-0.000661,True,304.746636,350.0,356.0,260.0,254.0,0.001170,0.001303,-0.000163,-0.000183,2.266090e-07,1.943390e-07,-4.549827e-09,-3.833463e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2022-09-13 12:00:00,293.680000,291.000000,-0.007727,-0.005138,False,300.590000,346.0,352.0,256.0,250.0,0.003638,0.001868,-0.001037,-0.000505,1.766507e-02,3.068726e-02,-3.645762e-03,-6.794046e-03
196,2022-09-13 13:00:00,291.000000,292.300000,0.005918,0.000780,False,300.590000,346.0,352.0,256.0,250.0,0.001720,0.002394,-0.000458,-0.000655,2.914387e-02,2.229472e-02,-6.353598e-03,-4.695618e-03
197,2022-09-13 14:00:00,292.270000,294.830000,0.006482,0.007262,False,300.590000,346.0,352.0,256.0,250.0,0.002191,0.004135,-0.000590,-0.001173,2.122483e-02,1.226893e-02,-4.401798e-03,-2.375079e-03
198,2022-09-13 15:00:00,294.830000,294.100000,-0.000364,0.006898,False,300.590000,346.0,352.0,256.0,250.0,0.003832,0.003201,-0.001072,-0.000882,1.152623e-02,1.352672e-02,-2.192495e-03,-2.624763e-03


In [14]:
ic_strat = IronCondors(long_offset=15, short_offset=15, qbw=qbw)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='week',
    update_freq='once')